# Descrizione del problema
Il problema riguarda lo **scheduling**. Sono dati $G$ ingegneri che lavorono per $L$ giorni. La realtà è composta da 3 elementi:
* **Features**: sono le funzionalità implementate nei prodotti;
* **Services**: sono i programmi che sono in esecuzione sui server;
* **Binaries**: modellano un server. Ogni *binary* può avere in esecuzione più servizi;

## Relazioni tra le entità
Una **Feature** i contraddistinta dal numero di utenti che riesce a soddisfare e da un insieme di servizi da cui dipende. Una **Feature** è disponibile ai clienti quando viene implementata in ogni servizio da cui dipende.<br>

Un **Service** è presente su **uno e un solo binary**, ma su ogni **binary** è possibile inserire più **Services**.

Ogni ingegnere $G_i$ può cominciare ad eseguire un task $T_j$ che può prendere un numero di giorni $D_k$. In particolare, possono essere eseguite le seguenti mosse:
- Implementare una **Feature**: su un *binary* $B_j$ può implementare tutti i servizi per una feature $F_i$. Questa mossa ha una durata $D_{Fi} + R_{Bj} + C_{Bj}$. Dove $D_{Fi}$ è data dalla complessità della **Feature**, $R_{Bj}$ è dato dal numero di servizi totali presenti nel *binary*, $C_{Bj}$ ingegneri che stanno già lavorando su $B_j$ nel giorno in cui viene *schedulata* l'attività. Inoltre, ogni ingegnere può implementare la stessa **Feature** su **binary** diversi. Infine, se più ingegneri sono *schedulati* per lavorare sullo stesso **binary** il primo della soluzione viene schedulato e conta per gli altri nel fattore $C_{Bj}$;

- Spostare un servizio: un ingegnere può spostare un servizio $S_i$ da un *binary* $B_j$ a $B_k$, mantenendo le funzionalità implementate. Per fare questa mossa, un ingegnere impiega $\max (R_{bj}, R_{bk})$ giorni dove $R_{bj}, R_{bk})$ sono i servizi in esecuzione rispettivamente su $B_j$ a $B_k$. Quest'azione però è bloccante: nessun ingegnere può lavorare su $B_j$ e $B_k$ quando è in corso una migrazione; 

- Creazione di un nuovo **Binary**: un ingegnere può creare un **Binary** nuovo senza nessun servizio in $N$ giorni;

- Idle: un ingegnere può non fare nulla e aspettare;

## Preprocessing
Dato il file in ingresso, le seguenti operazioni vengono svolte:
* la prima operazione consiste nel creare un dizionario in cui per ogni **binary** si memorizzano i servizi che sono installati su di esso;
* per ogni **feature** si crea un dizionario contente difficoltà, servizi di dipendenza e il numero di clienti che soddisfa;

In [70]:
fname = "data/a_example.in.txt"

# lettura dei dati dal file di input
with open(fname, "r") as f:
    lines = f.readlines()
    
# parsing della prima riga: metadati del problema
metadata = [int(k) for k in lines[0].strip().split(" ")]
num_days, num_engineers, num_services, num_binaries, num_features, days2new = metadata

# creazione dizionario dei binaries
binaries = {id: [] for id in range (num_binaries)}

service2binary = [line.strip().split() for line in lines[1:num_services+1] ]
for (service, binary)  in service2binary:
    binaries[int(binary)].append(service)

# creazione del dizionario features
features = {}
for i, feature in enumerate(lines[1+num_services:]):
    if i % 2 ==0:
        feature, services, difficulty, users = feature.strip().split(" ")
        services = lines[1+num_services+1+i].split()
        features[feature] = {
            "services": services,
            "users": int(users),
            "difficulty": int(difficulty)
        }


{0: ['se'], 1: ['sa', 'sc'], 2: ['sb', 'sd']}
{'foo': {'services': ['sc', 'sb', 'sd'], 'users': 100, 'difficulty': '3'}, 'bar': {'services': ['sc', 'sa'], 'users': 20, 'difficulty': '1'}}


# Approccio 1
In una fase preliminare, l'algoritmo ordina le feature per numero di utenti soddisfatti. In questo approccio, si assegna un punteggio maggiore ai servizi che risiedono sulla stessa macchina.